Классы

Client(id,name,surname,patronumic,gender,visit_countener,regular_client)

Deal(id,time,client,service)

Service(id,name,cost,gender)

In [1]:
from dataclasses import dataclass

import random
from faker import Faker
fake_data = Faker("ru_RU")
# import datetime 
import sqlite3
from sqlite3 import Error

import pandas as pd

### Запросы и подключение к БД

In [2]:
db_file = "salon.db"

sql_create_clients_table = """
                        CREATE TABLE IF NOT EXISTS clients (
                            id integer PRIMARY KEY AUTOINCREMENT,
                            name text NOT NULL,
                            surname text NOT NULL,
                            patronumic text NOT NULL,
                            gender bool NOT NULL,
                            visit_countener integer DEFAULT 1,
                            regular_client bool
                        ); """

sql_create_services_table = """
                        CREATE TABLE IF NOT EXISTS services (
                            id integer PRIMARY KEY AUTOINCREMENT,
                            name text NOT NULL,
                            cost integers NOT NULL,
                            gender bool NOT NULL
                        ); """

sql_create_deals_table = """
                        CREATE TABLE IF NOT EXISTS deals  (
                            id integer PRIMARY KEY AUTOINCREMENT,
                            client_id bigint,
                            service_id bigint,
                            time NOT NULL,
                            FOREIGN KEY (client_id) REFERENCES clients (id),
                            FOREIGN KEY (service_id) REFERENCES services (id)
                        ); """

sql_insert_clients_table = """ INSERT INTO clients (name, surname, patronumic, gender, visit_countener, regular_client) VALUES (?,?,?,?,?,?); """

sql_insert_services_table = """ INSERT INTO services (name, cost, gender) VALUES(?,?,?); """

sql_insert_deals_table = """ INSERT INTO deals (client_id, service_id, time) VALUES(?,?,?); """

sql_select_clients_table = """ SELECT * FROM clients; """

sql_select_services_table = """ SELECT * FROM services; """

sql_select_deals_table = """ SELECT * FROM deals; """

sql_update_client = """   UPDATE clients
                        SET name = ?,
                            surname = ?,
                            patronumic = ?,
                            gender = ?,
                            visit_countener = ?,
                            regular_client = ?
                        WHERE id = ? """

sql_update_service = """   UPDATE services
                        SET name = ?,
                            cost = ?,
                            gender = ?
                        WHERE id = ? """

sql_delete_client = """ DELETE FROM clients WHERE id = ? """

sql_delete_service = """ DELETE FROM services WHERE id = ? """

sql_select_client_by_id = """ SELECT * FROM clients WHERE id = ?; """

sql_select_service_by_id = """ SELECT * FROM services WHERE id = ?; """

### Создание таблицы

In [3]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        # print(sqlite3.version)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [4]:
conn = create_connection(db_file)


create_table(conn, sql_create_clients_table)
create_table(conn, sql_create_services_table)
create_table(conn, sql_create_deals_table)

### Функции коннекта объектов с базой данных 

In [5]:
def insert_clients_table(conn, obj):
    try:
        cur = conn.cursor()
        cur.execute(sql_insert_clients_table, (obj.name, obj.surname, obj.patronumic, obj.gender, obj.visit_countener, obj.regular_client))
        conn.commit()
        return cur.lastrowid
    except Error as e:
        print(e)

def insert_services_table(conn, obj):
    try:
        cur = conn.cursor()
        cur.execute(sql_insert_services_table, (obj.name, obj.cost, obj.gender))
        conn.commit()
        return cur.lastrowid
    except Error as e:
        print(e)

def insert_deals_table(conn, obj):
    try:
        cur = conn.cursor()
        cur.execute(sql_insert_deals_table, (obj.client_id, obj.service_id,obj.time))
        conn.commit()
        return cur.lastrowid
    except Error as e:
        print(e)


def update_client(conn, obj):
    try:
        cur = conn.cursor()
        cur.execute(sql_update_client, (obj.name, obj.surname, obj.patronumic, obj.gender, obj.visit_countener, obj.regular_client, obj._id))
        conn.commit()
    except Error as e:
        print(e)

def update_service(conn, obj):
    try:
        cur = conn.cursor()
        cur.execute(sql_update_service, (obj.name, obj.cost, obj.gender, obj._id))
        conn.commit()
    except Error as e:
        print(e)


def delete_client(conn, obj):
    try:
        cur = conn.cursor()
        cur.execute(sql_delete_client, (obj._id, ))
        conn.commit()
    except Error as e:
        print(e)

def delete_service(conn, obj):
    try:
        cur = conn.cursor()
        cur.execute(sql_delete_service, (obj._id, ))
        conn.commit()
    except Error as e:
        print(e)


def select_client_by_id(conn, id):
    try:
        cur = conn.cursor()
        cur.execute(sql_select_client_by_id, (id, ))
        conn.commit()
        return cur.fetchone()
    except Error as e:
        print(e)

def select_service_by_id(conn, id):
    try:
        cur = conn.cursor()
        cur.execute(sql_select_service_by_id, (id, ))
        conn.commit()
        return cur.fetchone()
    except Error as e:
        print(e)




### Классы

In [6]:
@dataclass
class Client:
    _id: int
    name: str
    surname: str
    patronumic: str
    gender: str 
    visit_countener: int 
    regular_client: bool 


    def __init__(self, name=None, surname=None, patronumic=None, gender=None, visit_countener=None, regular_client=None, get_by_id=None):
        if get_by_id == None:

            fake_name, fake_surname, fake_patronumic = fake_data.name().split(' ')

            if name == None:
                self.name = fake_name
            else:
                self.name = name
            
            if surname == None:
                self.surname = fake_surname 
            else:
                self.surname = surname

            if patronumic == None:
                self.patronumic = fake_patronumic
            else: 
                self.patronumic = patronumic
            
            if gender == None:
                self.gender = fake_data.boolean(chance_of_getting_true=50)
            else:
                self.gender = gender

            if visit_countener == None:
                self.visit_countener = random.randint(1, 6)
            else:
                self.visit_countener = visit_countener

            if regular_client == None:
                self.regular_client = fake_data.boolean(chance_of_getting_true=50)
            else:
                self.regular_client = regular_client            

            self._id = insert_clients_table(conn, self)
        else: # добавить проверку на корректность id
            obj = select_client_by_id(conn, get_by_id)
            self._id = obj[0]
            self.name = obj[1]
            self.surname = obj[2]
            self._patronumic = obj[4]
            self._gender = obj[5]
            self._visit_countener = obj[6]
            self._regular_client = obj[7]

    def increment_visit_countener(self):
        self._visit_countener += 1
        update_client(conn, self)


    def delete(self):
        delete_client(conn, self)
    

    def update(self, name=None, surname=None, patronumic=None, gender=None, visit_countener=None, regular_client=None):
        if name != None:
            self.name = name
        update_client(conn, self)    
        if name !=None:
            self.surname = surname
        update_client(conn, self)     
        if patronumic !=None:
            self.patronumic = patronumic
        update_client(conn, self)    
        if gender != None:
            self.gender = gender
        update_client(conn, self)    
        if visit_countener != None:
            self.visit_countener = visit_countener
        update_client(conn, self)    
        if regular_client != None:
            self.regular_client = regular_client
        update_client(conn, self)                        


@dataclass
class Service:
    _id: int
    name: str
    cost: int

    def __init__(self, name=None, cost=None, gender=None):
        if name == None:
            self.name = fake_data.color_name()
        else:
            self.name = name
        if cost == None:
            self.cost = random.randint(100, 1000)
        else:
            self.cost = cost
        if gender == None:
            self.gender = fake_data.boolean(chance_of_getting_true=50)
        else:
            self.gender = gender    
        self._id = insert_services_table(conn, self)


@dataclass
class Deal:
    _id: int
    client_id: int
    service_id: int
    time: str

    def __init__(self, client_id=None, service_id=None, time=None):
        if client_id == None:
            client_id_list = pd.read_sql(sql_select_clients_table, conn)['id'].tolist()
            self.client_id = random.choice(client_id_list)
        else:
            self.client_id = client_id
        
        if time == None:
            self.time = fake_data.date()
        else:
            self.time = time            
        
        if service_id == None:
            service_id_list = pd.read_sql(sql_select_services_table, conn)['id'].tolist()
            self.service_id = random.choice(service_id_list)
        else:
            self.service_id = service_id

        self._id = insert_deals_table(conn, self)


    def info(self):
        return (self, select_client_by_id(conn, self.client_id), select_service_by_id(conn, self.service_id))


In [7]:
c = Client()
c


Client(_id=1, name='Потапов', surname='Трофим', patronumic='Фадеевич', gender=False, visit_countener=2, regular_client=False)

In [8]:
s = Service()
s

Service(_id=1, name='Серебряный', cost=105)

In [9]:
d = Deal()
d

Deal(_id=1, client_id=1, service_id=1, time='2012-02-19')

In [10]:
pd.read_sql(sql_select_clients_table, conn)

,id,name,surname,patronumic,gender,visit_countener,regular_client
0,1,Потапов,Трофим,Фадеевич,0,2,0


In [11]:
pd.read_sql(sql_select_services_table, conn)

,id,name,cost,gender
0,1,Серебряный,105,0


In [12]:
pd.read_sql(sql_select_deals_table, conn)

,id,client_id,service_id,time
0,1,1,1,2012-02-19


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f982d86a-fb45-4a02-b4ea-c7285dcef6ac' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>